In [1]:
import joblib
import rasterio
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split

In [2]:
# Define index calculation functions
def calculate_savi(red, nir, L=0.5):
    return ((nir - red) * (1 + L)) / (nir + red + L)

def calculate_ndvi(red, nir):
    return (nir - red) / (nir + red)

def calculate_vari(green, red, blue):
    return (green - red) / (green + red - blue)

def calculate_exg(green, red, blue):
    return 2 * green - red - blue

def calculate_ndre(nir, red_edge):
    return (nir - red_edge) / (nir + red_edge)

# model_weight = "/blue/changzhao/zhou.tang/botanical_composition/data/bc_rf_model_20241104.joblib"
model_weight = "/blue/changzhao/zhou.tang/botanical_composition/data/bc_rf_model_20241105_allData.joblib"
rf_model = joblib.load(model_weight)

/blue/changzhao/zhou.tang/conda/envs/soc_gpu/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/blue/changzhao/zhou.tang/conda/envs/soc_gpu/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
tiff_src = "/blue/changzhao/zhou.tang/botanical_composition/data/All_Paddock_26_JUL_2024_ortho_bgrent.tiff"
# tiff_src = "/blue/changzhao/zhou.tang/botanical_composition/data/All_Paddock_22_SEP_2024_ortho_bgrent.tiff"
output_tiff_name = "/blue/changzhao/zhou.tang/botanical_composition/data/predictionmap_July_20241105.tiff"
chunk_size = 10000000
# Read the multi-band TIFF
with rasterio.open(tiff_src) as src:
    band_1_b = src.read(1)
    band_2_g = src.read(2)
    band_3_r = src.read(3)
    band_4_rd = src.read(4)
    band_5_nir = src.read(5)
    band_6_lwir = src.read(6)  # If needed for other analysis

# Calculate the indices
SAVI = calculate_savi(band_3_r, band_5_nir)
NDVI = calculate_ndvi(band_3_r, band_5_nir)
VARI = calculate_vari(band_2_g, band_3_r, band_1_b)
ExG = calculate_exg(band_2_g, band_3_r, band_1_b)
NDRE = calculate_ndre(band_5_nir, band_4_rd)
# savi_mask = SAVI > 0.9 # used for 20241104 prediction map
savi_mask = SAVI > 0.6   # used for 20241105 prediction map

print("stack features")
# Prepare the features for prediction
# Assume the model expects a 2D array with specific feature order, e.g., [SAVI, NDVI, VARI, EXG, NDRE]
features = np.stack([band_1_b, band_2_g, band_3_r, band_4_rd, band_5_nir, band_6_lwir, SAVI, NDVI, VARI, ExG, NDRE], axis=-1)

# Replace inf/-inf with NaN
features[np.isinf(features)] = np.nan
# Clip values to a reasonable range (optional, adjust as needed)
features = np.clip(features, -1e6, 1e6)
print(f"features shape is {features.shape}")

# Flatten the SAVI mask and feature array for consistent indexing
savi_mask_flat = savi_mask.flatten()
print(f"savi_mask_flat shape is {savi_mask_flat.shape}")
# Reshape the features array to be 2D: (num_pixels, num_features)
features_reshaped = features.reshape(-1, features.shape[-1])
print(f"features_reshaped shape is {features_reshaped.shape}")
# Filter out NaN values from features array
# Create a mask where none of the feature values are NaN or inf
valid_mask = ~np.isnan(features_reshaped).any(axis=1) & ~np.isinf(features_reshaped).any(axis=1)
# Combine valid_mask with the SAVI condition
combined_mask = savi_mask_flat & valid_mask
print(f"combined_mask shape is {combined_mask.shape}")

# Apply the combined mask to filter out invalid rows
features_valid = features_reshaped[combined_mask]
# Convert to DataFrame with appropriate feature names for prediction
feature_names = ["band_1_b", "band_2_g", "band_3_r", "band_4_rd", "band_5_nir", "band_6_lwir", "SAVI", "NDVI", "VARI", "ExG", "NDRE"]
features_valid_df = pd.DataFrame(features_valid, columns=feature_names)


# Split the valid features into chunks for processing
num_pixels = features_valid_df.shape[0]
num_chunks = (num_pixels // chunk_size) + 1 

# Create an empty array for storing prediction results
output_array = np.full(SAVI.shape, np.nan)
output_array_flat = output_array.flatten()

predictions_flat = np.full(features_valid_df.shape[0], np.nan)

class_mapping = {
    'grass': 1,
    'legume': 2
    # Add other classes as needed
}

for i in range(num_chunks):
    # Get start and end indices for the current chunk
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, num_pixels)
    
    # Extract the current chunk of features and mask
    features_chunk_df = features_valid_df.iloc[start_idx:end_idx,:]
    print(f"start predict from {start_idx} to {end_idx}")

    if features_chunk_df.size > 0:

        # Make predictions on the current chunk
        predictions_chunk = rf_model.predict(features_chunk_df)
        # print(predictions_chunk)
        predictions_chunk_mapped = np.array([class_mapping[pred] for pred in predictions_chunk])
        # print(predictions_chunk_mapped)
        # Assign predictions to the corresponding positions in the flattened output array
        predictions_flat[start_idx:end_idx] = predictions_chunk_mapped

        
output_array_flat[combined_mask] = predictions_flat
# Reshape the output array back to the original image shape
output_array = output_array_flat.reshape(SAVI.shape)

# Save the predictions as a new TIFF
with rasterio.open(
    output_tiff_name,
    'w',
    driver='GTiff',
    height=src.height,
    width=src.width,
    count=1,
    dtype='int32',
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(output_array, 1)

print("Prediction TIFF has been saved successfully.")

/scratch/local/49290172/ipykernel_3008085/3439913320.py:6: RuntimeWarning: invalid value encountered in divide
  return (nir - red) / (nir + red)
/scratch/local/49290172/ipykernel_3008085/3439913320.py:9: RuntimeWarning: divide by zero encountered in divide
  return (green - red) / (green + red - blue)
/scratch/local/49290172/ipykernel_3008085/3439913320.py:9: RuntimeWarning: invalid value encountered in divide
  return (green - red) / (green + red - blue)
/scratch/local/49290172/ipykernel_3008085/3439913320.py:15: RuntimeWarning: invalid value encountered in divide
  return (nir - red_edge) / (nir + red_edge)


stack features
features shape is (28914, 16933, 11)
savi_mask_flat shape is (489600762,)
features_reshaped shape is (489600762, 11)
combined_mask shape is (489600762,)
start predict from 0 to 10000000
start predict from 10000000 to 20000000
start predict from 20000000 to 30000000
start predict from 30000000 to 40000000
start predict from 40000000 to 50000000
start predict from 50000000 to 60000000
start predict from 60000000 to 70000000
start predict from 70000000 to 80000000
start predict from 80000000 to 90000000
start predict from 90000000 to 100000000
start predict from 100000000 to 110000000
start predict from 110000000 to 120000000
start predict from 120000000 to 130000000
start predict from 130000000 to 140000000
start predict from 140000000 to 150000000
start predict from 150000000 to 160000000
start predict from 160000000 to 170000000
start predict from 170000000 to 180000000
start predict from 180000000 to 190000000
start predict from 190000000 to 200000000
start predict from

/blue/changzhao/zhou.tang/conda/envs/soc_gpu/lib/python3.8/site-packages/numpy/core/_asarray.py:130: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)


Prediction TIFF has been saved successfully.


In [5]:
# Flatten the SAVI mask and feature array for consistent indexing
savi_mask_flat = savi_mask.flatten()
print(f"savi_mask_flat shape is {savi_mask_flat.shape}")
# Reshape the features array to be 2D: (num_pixels, num_features)
features_reshaped = features.reshape(-1, features.shape[-1])
print(f"features_reshaped shape is {features_reshaped.shape}")
# Filter out NaN values from features array
# Create a mask where none of the feature values are NaN or inf
valid_mask = ~np.isnan(features_reshaped).any(axis=1) & ~np.isinf(features_reshaped).any(axis=1)
# Combine valid_mask with the SAVI condition
combined_mask = savi_mask_flat & valid_mask
print(f"combined_mask shape is {combined_mask.shape}")

savi_mask_flat shape is (489600762,)
features_reshaped shape is (489600762, 11)
combined_mask shape is (489600762,)


In [6]:
# Apply the combined mask to filter out invalid rows
features_valid = features_reshaped[combined_mask]

# Convert to DataFrame with appropriate feature names for prediction
feature_names = ["band_1_b", "band_2_g", "band_3_r", "band_4_rd", "band_5_nir", "band_6_lwir", "SAVI", "NDVI", "VARI", "ExG", "NDRE"]
features_valid_df = pd.DataFrame(features_valid, columns=feature_names)

print(features_valid_df.shape[0])

412550134


In [10]:
chunk_size = 1000000
# Split the valid features into chunks for processing
num_pixels = features_valid_df.shape[0]
num_chunks = (num_pixels // chunk_size) + 1 

# Create an empty array for storing prediction results
output_array = np.full(SAVI.shape, np.nan, dtype='float32')
output_array_flat = output_array.flatten()

predictions_flat = np.full(features_valid_df.shape[0], np.nan)


class_mapping = {
    'grass': 1,
    'legume': 2
    # Add other classes as needed
}

for i in range(num_chunks):
    # Get start and end indices for the current chunk
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, num_pixels)
    
    # Extract the current chunk of features and mask
    features_chunk_df = features_valid_df.iloc[start_idx:end_idx,:]
    print(f"start predict from {start_idx} to {end_idx}")

    if features_chunk_df.size > 0:

        # Make predictions on the current chunk
        predictions_chunk = rf_model.predict(features_chunk_df)
        # print(predictions_chunk)
        predictions_chunk_mapped = np.array([class_mapping[pred] for pred in predictions_chunk])
        # print(predictions_chunk_mapped)
        # Assign predictions to the corresponding positions in the flattened output array
        predictions_flat[start_idx:end_idx] = predictions_chunk_mapped

        
output_array_flat[combined_mask] = predictions_flat
# Reshape the output array back to the original image shape
output_array = output_array_flat.reshape(SAVI.shape)

# Save the predictions as a new TIFF
with rasterio.open(
    'output_predictions.tiff',
    'w',
    driver='GTiff',
    height=src.height,
    width=src.width,
    count=1,
    dtype='int32',
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(output_array, 1)

print("Prediction TIFF has been saved successfully.")

start predict from 0 to 1000000
start predict from 1000000 to 2000000
start predict from 2000000 to 3000000
start predict from 3000000 to 4000000
start predict from 4000000 to 5000000
start predict from 5000000 to 6000000
start predict from 6000000 to 7000000
start predict from 7000000 to 8000000
start predict from 8000000 to 9000000
start predict from 9000000 to 10000000
start predict from 10000000 to 11000000
start predict from 11000000 to 12000000
start predict from 12000000 to 13000000
start predict from 13000000 to 14000000
start predict from 14000000 to 15000000
start predict from 15000000 to 16000000
start predict from 16000000 to 17000000
start predict from 17000000 to 18000000
start predict from 18000000 to 19000000
start predict from 19000000 to 20000000
start predict from 20000000 to 21000000
start predict from 21000000 to 22000000
start predict from 22000000 to 23000000
start predict from 23000000 to 24000000
start predict from 24000000 to 25000000
start predict from 250000

TypeError: invalid dtype: dtype('O')

In [11]:
# Save the predictions as a new TIFF
with rasterio.open(
    'output_predictions.tiff',
    'w',
    driver='GTiff',
    height=src.height,
    width=src.width,
    count=1,
    dtype='int32',
    crs=src.crs,
    transform=src.transform,
) as dst:
    dst.write(output_array, 1)

print("Prediction TIFF has been saved successfully.")

/blue/changzhao/zhou.tang/conda/envs/soc_gpu/lib/python3.8/site-packages/numpy/core/_asarray.py:130: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)


Prediction TIFF has been saved successfully.
